# Localización ciudades nacimiento autores.

**A continuación, obtendremos los distintos lugares de nacimiento de cada uno de los artistas mediante un web scraping.**

❗ **Además, geolocalizaremos las ciudades y las posicionaremos en un mapa.** 

In [1]:
import requests
import pandas as pd
import folium
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

import sys
sys.path.append('../')
import src.funcionesciudades as fc

**En primer lugar, creamos un diccionario donde almacenar los datos scrapeados de la ciudad y el pais de cada autor.**

In [2]:
ciudades_natales = {'ciudad': [], "pais": []}

In [3]:
art = pd.read_csv('../data/art.csv', index_col = 0)
art.head(2)

,name,years,genre,nationality,bio,wikipedia,paintings,year_of_birth,year_of_death
id,,,,,,,,,
0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani was an Italian Jew...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193,1884,1920
1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky was a Russian...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88,1866,1944


**En el csv hay una columna que contiene el enlace de la página web de la wikipedia de cada artista, por lo que iterando por esta columna es posible acceder a la información de cada uno de los artistas.**

Sin embargo, nos encontramos con distintos problemas:

1️⃣ En algunos casos, no hay información sobre el lugar de nacimiento.

2️⃣ Por otro lado, otros autores únicamente contienen información de la ciudad.

Por ello, creamos una variable (x) que contenga la información escrapeada con los lugares de nacimiento y, según su longitud, apendearemos unos resultados u otros. Así, si la longitud de x es igual a dos apendeamos ambos datos, si es uno apendeamos la ciudad y, en caso de no cumplir ninguna condición, apendeamos que no tenemos información disponible.

In [6]:
links = art['wikipedia']

In [7]:
for url in links:
    try:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        nacimientos = soup.find_all("div",{"class": "birthplace"})
        x = nacimientos[0].find_all('a')

        if len(x) == 2: 
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append(x[1].get('title'))
        else:
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append('Información no disponible.')  
    except:
        ciudades_natales['ciudad'].append('Información no disponible.')
        ciudades_natales['pais'].append('Información no disponible.')


🏡 **De este modo, obtenemos un dataframe con los lugares de nacimiento de cada artista.**

In [8]:
lugar_nacimiento = pd.DataFrame(ciudades_natales)
lugar_nacimiento.head(2)

,ciudad,pais
0,Livorno,Kingdom of Italy
1,Moscow,Russian Empire


🌍 **Una vez obtenida la ciudad, procedemos a obtener la longitud y latitud de la misma a través de una función con la ayuda de geopy.**

Para ello, en primer lugar obtenemos ambas y posteriormente creamos un dataframe nuevo donde la longitud sea una columna y la latitud sea otra. Esto es necesario para poder obtener el mapa.

In [9]:
ciudades = ciudades_natales['ciudad']

In [10]:
sc = fc.sacar_coord

In [11]:
geo_ciudad = pd.DataFrame(lugar_nacimiento["ciudad"].apply(sc))

In [12]:
geo_ciudad.head(5)

,ciudad
0,"(42.790216599999994, 10.340278678735025)"
1,"(55.7504461, 37.6174943)"
2,"(19.3834636, -99.2190861)"
3,"(48.8588897, 2.3200410217200766)"
4,"(50.7123036, 3.8301131)"


In [13]:
slat = fc.sacar_latitud

In [14]:
geo_ciudad["latitud"] = geo_ciudad["ciudad"].apply(slat) 

In [15]:
slong = fc.sacar_longitud

In [16]:
geo_ciudad["longitud"] = geo_ciudad["ciudad"].apply(slong) 

❗ **Es importante eliminar los valores nulos de los autores de los que no tenemos información para evitar fallos en el empleo de folium.**

In [19]:
geo_def = geo_ciudad.dropna()

🗺 **Por último, obtenemos el mapa localizando cada ciudad a través de su longitud y latitud.**

In [20]:
mapa=folium.Map(location=[40, 15],
                tiles='openstreetmap',
                zoom_start=2)

for index, row in geo_def.iterrows():
    folium.Marker([row["latitud"], row['longitud']],
                  radius=10,
                  icon=folium.Icon(icon = "globe", icon_color = 'black', color = "lightgreen")).add_to(mapa)

mapa

🤓🧐 En conclusión, como podemos observar en el mapa, la mayoría de autores nacieron en Europa, destacando especialmente el norte de Italia y los Países Bajos.